In [1]:
import os
import platform

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# import matplotlib.pyplot as plt
import tqdm
import cv2
import pandas as pd

from street_signs_identify.utils.x_image_loader import XImageLoader
import street_signs_identify.utils.check_overlap as co

from street_signs_identify.detector.fr_detector import FRCNNDetector

In [2]:
folder_name = "val"
data_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{folder_name}'
output_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xf\\{folder_name}'
output_folder_gt = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xf_gt\\{folder_name}'
output_folder_filtered_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xf_filter\\{folder_name}'
output_folder_craft_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xf_craft\\{folder_name}'

if platform.system() == "Linux":
    folder_name = "./dataset/"
    data_path = os.path.join(folder_name, "2Penghu")
    output_folder = os.path.join(folder_name, "cropped_2Penghu")
    output_folder_gt = os.path.join(output_folder, "ref_gt")
    output_folder_filtered_pred = os.path.join(output_folder, "filtered_pred")
    output_folder_craft_pred = os.path.join(output_folder, "craft_pred")


os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder_gt, exist_ok=True)
os.makedirs(output_folder_filtered_pred, exist_ok=True) 
os.makedirs(output_folder_craft_pred, exist_ok=True)

In [3]:
classes = ['1', '2', '3', '4', '5', 'blue', 'brown', 'green', 'red', 'yellow', 'parking', 'limit_h', 'limit_speed' ,'14', '15']
detector = FRCNNDetector(classes=classes)
x_image_loader = XImageLoader(data_path, annotation_file="labels.csv")

In [4]:
x_image_loader._info.head()

,raw_filename,filename,category,box,label,polygon,x0,y0,x1,y1
0,221005_014013216_Camera_0.jpg,1Tainan_nan,1,"[(787, 809), (784, 857), (980, 851), (977, 798)]",沙崙綠能科技示範場域,"POLYGON ((787 809, 784 857, 980 851, 977 798, ...",784,798,980,857
1,221005_014013216_Camera_0.jpg,1Tainan_nan,1,"[(980, 899), (786, 908), (784, 857), (977, 850)]",中央研究院南部院區,"POLYGON ((980 899, 786 908, 784 857, 977 850, ...",784,850,980,908
2,221005_014013216_Camera_0.jpg,1Tainan_nan,1,"[(977, 950), (788, 960), (786, 908), (981, 900)]",國立陽明交通大學臺南分部,"POLYGON ((977 950, 788 960, 786 908, 981 900, ...",786,900,981,960
3,221005_014013216_Camera_0.jpg,1Tainan_nan,1,"[(981, 848), (982, 898), (1188, 897), (1191, 8...",資安暨智慧科技研發大樓,"POLYGON ((981 848, 982 898, 1188 897, 1191 849...",981,848,1191,898
4,221005_014013216_Camera_0.jpg,1Tainan_nan,1,"[(982, 950), (1182, 947), (1181, 896), (982, 8...",臺灣智駕測試實驗室,"POLYGON ((982 950, 1182 947, 1181 896, 982 899...",982,896,1182,950


In [6]:
global_index = 0

label_df = pd.DataFrame(columns=["filename", "category", "words", "src_file", "IOU"])

for image_name in tqdm.tqdm(x_image_loader._image_names):
    detector.eval()
    gt_image = x_image_loader[image_name]
    detected_image = detector(gt_image.image)
    if detected_image is None:
        continue
    
    overlapping_info = co.check_overlapping_area(gt_image.info, detected_image.info)
    if overlapping_info is None:
        continue

    # save gt plot
    gt_img = gt_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_gt, image_name), cv2.cvtColor(gt_img, cv2.COLOR_RGB2BGR))
    
    # save craft plot
    det_img = detected_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_craft_pred, image_name), cv2.cvtColor(det_img, cv2.COLOR_RGB2BGR))

    detected_image = detected_image.keeps_index(overlapping_info["index_2nd"])

    filtered_det_img = detected_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_filtered_pred, image_name), cv2.cvtColor(filtered_det_img, cv2.COLOR_RGB2BGR))

    for row_idx, row in overlapping_info.iterrows():
        gt_index, detected_index = row["index_1st"], row["index_2nd"]
        detected_instance = detected_image[detected_index]
        if not detected_instance.legal:
            continue
        
        label_df.loc[global_index, "filename"] = f"{global_index}.jpg"
        label_df.loc[global_index, "src_file"] = image_name
        label_df.loc[global_index, "category"] = gt_image.info.loc[gt_index, "category"]
        label_df.loc[global_index, "words"] = gt_image.info.loc[gt_index, "label"]
        label_df.loc[global_index, "IOU"] = row["IOU"]

        crapped_img = detected_instance.image
        cv2.imwrite(os.path.join(output_folder, f"{global_index}.jpg"), cv2.cvtColor(crapped_img, cv2.COLOR_RGB2BGR))
        global_index+=1
        

label_df.to_csv(os.path.join(output_folder, "labels.csv"))

  0%|                                                                                           | 0/99 [00:00<?, ?it/s]


AttributeError: 'numpy.ndarray' object has no attribute 'dim'